# Paper

In [2]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import torch.nn as nn
import tqdm
import torch
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from cvar_sensing.utils import prepare_time_series, batch_interp_nd
import torch

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.metrics import AUC

2024-12-27 16:46:22.006727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-27 16:46:22.019497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-27 16:46:22.023392: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 16:46:22.034641: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-27 16:46:24.727508: W tensorflow/compiler/tf2

In [3]:
X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

train_y = np.eye(np.max(y_train) + 1)[y_train]
y_test = np.eye(np.max(y_test) + 1)[y_test]

train_x = np.concatenate([X_train_digits, X_train_counter], axis=1)
X_test = np.concatenate([X_test_digits, X_test_counter], axis=1)

In [4]:
classifier = XGBClassifier(n_estimators=256, device='gpu')
classifier.load_model('/work/users/d/d/ddinh/aaco/models/synthetic_raw.model')

In [5]:
# todo: implement progressively acquire features version 

In [ ]:
from sklearn.model_selection import train_test_split

class EmbeddingModels(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.num_features = 10 * 2
        self.base_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.num_features,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
        ])
        self.embedding_head = tf.keras.layers.Dense(self.num_features)

    def call(self, x):
        x = self.base_model(x)
        embedding_space = self.embedding_head(x)
        return embedding_space,x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'num_features': self.num_features,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


@tf.function
def embedding_loss(distribution_true, distribution_embedding):
    return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(distribution_true, distribution_embedding)

@tf.function
def prediction_loss(y_true, y_pred):
    return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)

def x_masked(x, d=20, num_timestamps=10, num_modalities=2):
    masks = np.concatenate(
        [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(int(d/3))], 1, keepdims=True) for _ in range(x.shape[0])], 1
        )
    masks = np.float32(masks.T)

    # ensure there is no 1 at the end of time step 
    for i in range(num_modalities):
        masks[:, num_timestamps * i + num_timestamps - 1] = 0
        
    masks_zero = np.sum(masks, axis=1) == 0
    
    
    while np.sum(masks_zero) > 0:
        # masks[masks_zero] = np.float32(np.concatenate(
        #     [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(np.sum(masks_zero))], 1
        #     )).T
        masks[masks_zero] = np.float32(np.concatenate(
            [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(int(d/3))], 1, keepdims=True) for _ in range(np.sum(masks_zero))], 1
            )).T
        for i in range(num_modalities):
            masks[:, num_timestamps * i + num_timestamps - 1] = 0
            
        masks_zero = np.sum(masks, axis=1) == 0
        
    return np.copy(x), masks


def compute_loss_timestep(y_true, x_data, mask, classifier, acquisition_cost, loss_function, num_timestamps=10, num_modalities=2):
    y_pred = classifier.predict_proba(np.concatenate([x_data * mask, mask], axis=1))
    y_pred = torch.Tensor(y_pred)
    if isinstance(y_true, tf.Tensor):
        y_true = torch.Tensor(y_true.numpy())
    else:
        y_true = torch.Tensor(y_true)
    total_cost = torch.zeros(mask.shape[0])
    total_cost += mask.sum(axis=1) * acquisition_cost
    total_loss = loss_function(y_pred, y_true).numpy() + total_cost.numpy()
    return total_loss

def get_potential_features(x, y, classifier, prev_masks, acquisition_cost, d=20, num_masks=1500, topk=5, num_timestamps=10, num_modalities=2):
    new_masks = np.concatenate(
        [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(num_masks)], 1
        )
    new_masks = np.float32(new_masks.T)

    # repeat for parallelization
    x_rep = np.repeat(x, num_masks, axis=0)
    y_rep = np.repeat(y, num_masks, axis=0)
    new_masks = np.concatenate([new_masks for _ in range(x.shape[0])], 0)
    prev_masks_rep = np.repeat(prev_masks, num_masks, axis=0)

    # combine previous masks with new masks
    N = x_rep.shape[0]
    segments_previous = prev_masks_rep.reshape(N, num_modalities, num_timestamps)
    segments_after = new_masks.reshape(N, num_modalities, num_timestamps)
    last_indices = np.where(segments_previous == 1, np.arange(num_timestamps), -1)
    last_indices_per_segment = np.max(last_indices, axis=2)  
    final_last_index = np.max(last_indices_per_segment, axis=1)  
    mask = np.arange(num_timestamps)[None, None, :] > final_last_index[:, None, None]
    final_segments = np.where(mask, segments_after, segments_previous)
    final = final_segments.reshape(N, num_timestamps * num_modalities)
    
    # compute loss for current and future masks
    current_loss = compute_loss_timestep(y, x, prev_masks, classifier, acquisition_cost, nn.CrossEntropyLoss(reduction='none'))
    future_loss = compute_loss_timestep(y_rep, x_rep, final, classifier, acquisition_cost, nn.CrossEntropyLoss(reduction='none'))
    
    # get the top k min loss
    top_k_sets = []
    distributions = []
    for i in range(x.shape[0]):
        min_current_loss = current_loss[i]
        single_sample_future_loss = np.copy(future_loss[i * num_masks: (i + 1) * num_masks])
        single_sample_future_set = np.copy(final[i * num_masks: (i + 1) * num_masks])
        
        top_k_loss = np.argsort(single_sample_future_loss)
        min_k_future_loss = single_sample_future_loss[top_k_loss]
        min_k_subsets = single_sample_future_set[top_k_loss]
        
        unique_min_k_subsets, unique_indices = np.unique(min_k_subsets, axis=0, return_index=True)
        unique_min_k_future_loss = min_k_future_loss[unique_indices]
        
        sorted_unique_indices = np.argsort(unique_min_k_future_loss)
        unique_min_k_future_loss = unique_min_k_future_loss[sorted_unique_indices]
        unique_min_k_subsets = unique_min_k_subsets[sorted_unique_indices]
        
        topk_min = min(topk, unique_min_k_subsets.shape[0])
        # e.g., topk = 5, while topk_min = 3
        # import pdb; pdb.set_trace()
        for j in range(topk_min):
            if np.equal(unique_min_k_subsets[j], prev_masks[i]).all():
                if topk_min == unique_min_k_subsets.shape[0]:
                    unique_min_k_subsets[j:topk_min-1] = unique_min_k_subsets[j+1:topk_min]
                    unique_min_k_future_loss[j:topk_min-1] = unique_min_k_future_loss[j+1:topk_min]
                    topk_min -= 1
                else: 
                    unique_min_k_subsets[j:topk_min] = unique_min_k_subsets[j+1:topk_min+1]
                    unique_min_k_future_loss[j:topk_min] = unique_min_k_future_loss[j+1:topk_min+1]
                break         
        unique_min_k_subsets = unique_min_k_subsets[:topk_min]
        subset_losses = unique_min_k_future_loss[:topk_min]
        unique_min_k_subsets[:,:d] -= prev_masks[i]
        
        # get the first 1 in each segment
        # todo: consider changing this into a for loop, might be faster
        n = unique_min_k_subsets.shape[0]
        segment_size = num_timestamps
        num_segments = num_modalities
        segments = unique_min_k_subsets.reshape(n, num_segments, segment_size)
        first_one_indices = np.where(segments == 1, np.arange(segment_size), np.inf)
        first_one_per_segment = np.min(first_one_indices, axis=2)
        first_one_per_segment[first_one_per_segment == np.inf] = -1
        global_first_one = np.min(
            np.where(first_one_per_segment != -1, first_one_per_segment, np.inf), axis=1
        )
        global_first_one = np.where(global_first_one == np.inf, -1, global_first_one)
        keep_indices = (first_one_indices == global_first_one[:, None, None])
        transformed_masks = np.zeros_like(unique_min_k_subsets)
        transformed_masks[keep_indices.reshape(unique_min_k_subsets.shape)] = 1
        
        unique_min_k_subsets = np.copy(transformed_masks)

        # turn into probabilities
        unique_min_k_subsets = unique_min_k_subsets / np.sum(unique_min_k_subsets, axis=1)[:, None]
        subset_losses = subset_losses[:topk_min]
        weights = np.exp(-subset_losses[:topk_min])
        weights /= np.sum(weights)
        distribution = np.sum(unique_min_k_subsets * weights[:, None], axis=0) 
        distributions.append(distribution)

    distributions = np.array(distributions) # shape (batch_size, d)
    distributions /= (distributions.sum(axis=1, keepdims=True) + 1e-12) # normalize for numerical stability
    # print("check sum of distributions: ", np.sum(distributions, axis=1))
    
    
    return top_k_sets, distributions
    
# @tf.function
def train_step(model, optimizer, x, y, classifier, acquisition_cost, alpha=1):
    x, prev_masks = x_masked(x)
    top_k_sets, distributions = get_potential_features(np.copy(x), np.copy(y), classifier, np.copy(prev_masks), acquisition_cost)

    with tf.GradientTape() as tape:
        embedding_space, embedding = model(x*prev_masks)
        loss = embedding_loss(distributions, embedding_space)
    print("TRAIN Mean of distributions: ", np.mean(distributions, axis=0), np.mean(distributions, axis=0).sum())

    # Backprop
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

def validation_step(model, x, y, classifier, acquisition_cost, alpha=1):
    x, prev_masks = x_masked(x)

    _, distributions = get_potential_features(
        x, y, classifier, np.copy(prev_masks), acquisition_cost
    )
    
    embedding_space, embedding = model(x * prev_masks)
    # print mean of next_feature_onehot
    print("VAL Mean of distributions: ", np.mean(distributions, axis=0), np.mean(distributions, axis=0).sum())

    loss = embedding_loss(distributions, embedding_space) 
    return loss


X, X_val, y, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(10000).batch(512).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

model = EmbeddingModels()
optimizer = tf.keras.optimizers.Adam(1e-3)

acquisition_cost = 0.015
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    epoch_loss = .00
    step_count = 0

    for batch_x, batch_y in dataset:
        loss = train_step(model, optimizer, batch_x, batch_y, classifier, acquisition_cost)
        epoch_loss += loss.numpy()
        step_count += 1
    train_loss = epoch_loss / step_count

    val_loss_sum = 0.0
    val_steps = 0
    for val_x_batch, val_y_batch in val_dataset:
        loss_val = validation_step(model, val_x_batch, val_y_batch, classifier, acquisition_cost)
        val_loss_sum += loss_val.numpy()
        val_steps += 1
    val_loss = val_loss_sum / val_steps
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save('/work/users/d/d/ddinh/aaco/models/embeddingembedding_synthetic_raw_gt.h5')
        model.save('/work/users/d/d/ddinh/aaco/models/embedding_synthetic_raw.keras')
        model.save_weights('/work/users/d/d/ddinh/aaco/models/embedding_synthetic_raw.weights.h5')

print("Training Complete.")
# add the regularizer same as divdis to avoid always predic as 0

TRAIN Mean of distributions:  [0.         0.01504594 0.02696427 0.01620061 0.03303646 0.04066233
 0.05919062 0.09902064 0.09062582 0.16383073 0.         0.01777053
 0.01248456 0.02245168 0.01695786 0.03867469 0.03180838 0.05642885
 0.09675755 0.16208862] 1.0000001
TRAIN Mean of distributions:  [0.         0.01151191 0.0152652  0.01466489 0.0382714  0.03377279
 0.05822695 0.08498827 0.09683277 0.17846382 0.         0.0076105
 0.00896783 0.02441586 0.02929544 0.02441479 0.03786929 0.05210138
 0.10921884 0.17410803] 0.99999994
TRAIN Mean of distributions:  [0.         0.0127041  0.01991013 0.01697238 0.03614616 0.03632533
 0.05002668 0.06407223 0.12054139 0.18072885 0.         0.00996682
 0.0093719  0.0216985  0.01971501 0.02032173 0.03203081 0.05035679
 0.11724312 0.18186806] 1.0
TRAIN Mean of distributions:  [0.         0.00488055 0.02306582 0.01559387 0.03129894 0.03223277
 0.04473104 0.08946662 0.09819248 0.19838142 0.         0.00215103
 0.01444041 0.01951774 0.01640397 0.02519753 0.

2024-12-27 17:28:58.826520: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


VAL Mean of distributions:  [0.         0.0160315  0.01270308 0.01583047 0.03162373 0.04374902
 0.0572294  0.09175565 0.09810881 0.1654265  0.         0.01053806
 0.01152524 0.02245704 0.0242359  0.04100779 0.03689949 0.05856623
 0.09093234 0.1713798 ] 1.0000001
VAL Mean of distributions:  [0.         0.01172315 0.01741942 0.01488195 0.03574162 0.03027393
 0.05683806 0.08947387 0.08728607 0.18669243 0.         0.01209962
 0.01308048 0.02068447 0.01621114 0.03533775 0.04001712 0.0550431
 0.09815432 0.17904145] 0.99999994
VAL Mean of distributions:  [0.         0.01290672 0.02404725 0.01855102 0.03654319 0.03593713
 0.05956417 0.08206704 0.09668941 0.1820031  0.         0.00779599
 0.01658586 0.02438827 0.01894265 0.02109343 0.04978206 0.05466397
 0.07980915 0.17862959] 0.99999994
VAL Mean of distributions:  [0.         0.01212677 0.02574956 0.01817713 0.02713513 0.03185358
 0.0597607  0.07268558 0.10799244 0.18520947 0.         0.00858853
 0.02149687 0.01580546 0.02012236 0.02714773 0.0

2024-12-27 17:39:22.532283: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


VAL Mean of distributions:  [0.         0.00992823 0.02133794 0.01471875 0.03935344 0.04483223
 0.07210679 0.08606181 0.10920826 0.15812078 0.         0.00992029
 0.0191051  0.01028725 0.0231485  0.02941578 0.04037813 0.0624437
 0.09952663 0.15010636] 0.99999994
Epoch 1, Train Loss: 2.7592, Validation Loss: 2.4469
TRAIN Mean of distributions:  [0.         0.01776709 0.01915766 0.01974052 0.03146157 0.04357047
 0.05450695 0.06874923 0.11539999 0.17165981 0.         0.00994826
 0.01834567 0.02363474 0.01345814 0.03144269 0.03573859 0.05348575
 0.09820205 0.17373079] 0.99999994
TRAIN Mean of distributions:  [0.         0.01836079 0.01488288 0.01873366 0.03731176 0.03437837
 0.05702524 0.08458871 0.10291146 0.16642962 0.         0.01247416
 0.0093609  0.0230624  0.018157   0.02811928 0.04609537 0.06308956
 0.10442425 0.16059464] 1.0
TRAIN Mean of distributions:  [0.         0.01660905 0.01701502 0.01721178 0.04239145 0.02967058
 0.04826135 0.06737358 0.10894589 0.19039191 0.         0.0138

...

VAL Mean of distributions:  [0.         0.02096884 0.02761084 0.02650723 0.05181606 0.0257328
 0.05440633 0.09452606 0.10956451 0.15606962 0.         0.01653176
 0.01723734 0.01839249 0.02312412 0.02794143 0.02941988 0.05327237
 0.09840611 0.14847225] 1.0000001
Epoch 11, Train Loss: 1.8173, Validation Loss: 1.7956
TRAIN Mean of distributions:  [0.         0.00761399 0.00880833 0.01894248 0.03596089 0.04494758
 0.06503035 0.08633933 0.11818099 0.17773342 0.         0.01035185
 0.00760291 0.01424507 0.01953297 0.02929425 0.03299294 0.05461121
 0.09737356 0.17043781] 0.99999994
TRAIN Mean of distributions:  [0.         0.01580933 0.02539285 0.01912841 0.0364889  0.04006362
 0.05586233 0.08066491 0.09931102 0.16952369 0.         0.01271948
 0.01443078 0.02227254 0.02208143 0.02599224 0.03672629 0.05130648
 0.10104713 0.17117858] 1.0
TRAIN Mean of distributions:  [0.         0.01133515 0.01208368 0.01129783 0.02557852 0.03282993
 0.05234786 0.07109519 0.10748774 0.19820969 0.         0.0074

2024-12-28 06:35:50.454267: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


TRAIN Mean of distributions:  [0.         0.01287934 0.02639648 0.01386777 0.04178669 0.03691261
 0.06233903 0.08006707 0.09474076 0.17630026 0.         0.00976552
 0.01619317 0.01504858 0.02108812 0.02362039 0.04590099 0.05664742
 0.09152368 0.17492221] 1.0000001
TRAIN Mean of distributions:  [0.         0.01153535 0.01798845 0.01561404 0.03672806 0.04611479
 0.06448366 0.07134284 0.09360802 0.17549111 0.         0.00877466
 0.00975068 0.02421635 0.02149172 0.0382693  0.04452096 0.05445535
 0.09336406 0.17225058] 0.99999994
TRAIN Mean of distributions:  [0.         0.02071963 0.00877297 0.01876014 0.04023339 0.03848011
 0.05546968 0.08045364 0.09665314 0.1728317  0.         0.01092759
 0.00408831 0.02501504 0.02538636 0.03846613 0.03752209 0.0519543
 0.09813967 0.17612615] 1.0
TRAIN Mean of distributions:  [0.         0.00916947 0.01447185 0.01523135 0.040624   0.03047813
 0.05194102 0.1041299  0.09705787 0.18102232 0.         0.00995291
 0.01053559 0.01721106 0.02067624 0.03204148 0.

VAL Mean of distributions:  [0.         0.0158001  0.01325963 0.02167963 0.03344676 0.04709149
 0.06431863 0.07431597 0.12762125 0.16693987 0.         0.01065738
 0.01542995 0.02244454 0.01212956 0.02131109 0.02903049 0.05507228
 0.10674515 0.16270624] 1.0
Epoch 21, Train Loss: 1.7968, Validation Loss: 1.7944
TRAIN Mean of distributions:  [0.         0.01288747 0.02467039 0.01933051 0.03304497 0.03907088
 0.06170966 0.05765602 0.1017762  0.18843038 0.         0.01012145
 0.01405611 0.02789613 0.01464753 0.02968351 0.03240649 0.0435124
 0.09890816 0.19019185] 1.0000001
TRAIN Mean of distributions:  [0.         0.01699836 0.02755431 0.01640012 0.03906957 0.04006825
 0.05292387 0.07109852 0.11390141 0.17669551 0.         0.00956721
 0.0166176  0.02186645 0.01599262 0.02908315 0.03107068 0.04525441
 0.0941956  0.1816424 ] 1.0
TRAIN Mean of distributions:  [0.         0.0207065  0.01444802 0.01464648 0.04041622 0.0375036
 0.05041361 0.09632578 0.09785917 0.18334946 0.         0.01444697
 0.

VAL Mean of distributions:  [0.         0.01469808 0.02577713 0.01544426 0.03455923 0.03636438
 0.06251841 0.08609525 0.10973193 0.1822844  0.         0.01102709
 0.0220549  0.0213196  0.0110217  0.01803076 0.0367519  0.05507036
 0.08292801 0.17432262] 1.0
Epoch 24, Train Loss: 1.7977, Validation Loss: 1.7903
TRAIN Mean of distributions:  [0.         0.01581434 0.016392   0.01153538 0.03952199 0.03554167
 0.04727307 0.06312706 0.0980662  0.2010769  0.         0.00919469
 0.01364344 0.018141   0.01872282 0.0343714  0.04295314 0.04233468
 0.09366226 0.19862798] 1.0
TRAIN Mean of distributions:  [0.         0.01678799 0.0230552  0.01481902 0.03866177 0.03340815
 0.05938439 0.07262014 0.0972052  0.18525508 0.         0.0101602
 0.01758219 0.01837531 0.02383736 0.02756036 0.03515156 0.05346201
 0.09599352 0.17668054] 1.0
TRAIN Mean of distributions:  [0.         0.01368603 0.02091168 0.01270923 0.02756852 0.04218826
 0.05702018 0.08088882 0.09468239 0.17769454 0.         0.0097512
 0.019695

In [6]:
model.save('/work/users/d/d/ddinh/aaco/models/embedding_gt.h5')
model.save('/work/users/d/d/ddinh/aaco/models/embedding_gt.keras')
model.save_weights('/work/users/d/d/ddinh/aaco/models/embedding_gt.weights.h5')

In [ ]:
NUM_MASKS_CURRENT = 100
NUM_MASKS_FUTURE = 100
NUM_TS = 12
NUM_MODALITIES = 4
acquisition_cost = 0.1
d = NUM_TS * NUM_MODALITIES
loss_function = nn.CrossEntropyLoss(reduction='none')

print("computing masks...")

ts_current = np.random.randint(0, num_ts, NUM_MASKS_CURRENT)
b_c = []
for i in range(NUM_MASKS_CURRENT):
    mask_temp = []
    for j in range(NUM_MODALITIES): 
        mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,:ts_current[i]+1][:, :np.random.randint(ts_current[i]+1)], 1)
        mask_temp.append(mask)
    b_c.append(mask_temp)
b_c = np.array(b_c)
b_c = np.transpose(b_c, (0, 2, 1)).reshape(-1, d)
b_empty = np.sum(b_c, 1) == 0
b_c = b_c[~b_empty]
b_c = np.unique(b_c, axis=0)
num_mask = b_c.shape[0]


x_current = np.repeat(train_x, num_mask, axis=0)
y_current = np.repeat(train_y, num_mask, axis=0)

b_current = np.concatenate([b_c for _ in range(train_x.shape[0])], axis=0)
print("computing predictions...")

y_pred = np.zeros(y_current.shape)
for ts in range(NUM_TS):
    print(ts)
    x_input = np.zeros(x_current.shape)
    mask_input = np.zeros(b_current.shape)
    
    for k in range(NUM_MODALITIES):
        x_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(x_current[:,k * NUM_TS: k * NUM_TS + ts + 1])
        mask_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(b_current[:,k * NUM_TS: k * NUM_TS + ts + 1])
    
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input.numpy())
    ts_rep = np.repeat(ts, x_input.shape[0]).reshape(-1, 1)
    pred = classifier.predict(np.concatenate([x_input * mask_input, ts_rep], axis=-1), verbose=0)
    y_pred[:,ts,:] = pred
y_pred = torch.Tensor(y_pred)
y_current = torch.Tensor(y_current)
print("computing loss...")
cost = torch.zeros(b_current.shape[0])
for m in range(NUM_MODALITIES):
    modality_cost = 1 if m in [0, 1] else 0.5
    cost += b_current[:, m * NUM_TS: (m + 1) * NUM_TS].sum(1) * acquisition_cost * modality_cost
loss_current = compute_accumulated_loss(y_pred, y_current, loss_function) + cost 

computing masks...
computing predictions...
0
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [ ]:
print("computing best mask...")
# while loop here to make sure mask is not 0
Bf = []
for i in range(NUM_MASKS_CURRENT):
    for j in range(NUM_MASKS_FUTURE): 
        mask_temp = []
        for j in range(NUM_MODALITIES): 
            mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,ts_current[i]+1:][:, :np.random.randint(NUM_TS-ts_current[i]+1)], 1)
            mask_temp.append(mask)
        Bf.append(mask_temp)
Bf = np.array(Bf)
Bf = np.transpose(Bf, (0, 2, 1)).reshape(-1, d)
    
Bf[0] = np.zeros(d) # current mask 
Bf = np.unique(Bf, axis=0)

num_mask_future = Bf.shape[0]
 
x_future = np.repeat(x_current, num_mask_future, axis=0)
y_future = np.repeat(y_current, num_mask_future, axis=0)
print("x_future shape: ", x_future.shape)

b_current_for_future = np.repeat(b_c, num_mask_future, axis=0)


computing best mask...
x_future shape:  (6355800, 48)


In [8]:
Bforb = np.repeat(Bf, num_mask, axis=0)

Bforb = np.minimum(b_current_for_future+Bforb, 1)
Bforb = np.concatenate([Bforb for _ in range(train_x.shape[0])], axis=0)

In [13]:
print("computing predictions...")
y_pred = np.zeros(y_future.shape)
for ts in range(NUM_TS):
    print(ts)
    x_input = np.zeros(x_future.shape)
    mask_input = np.zeros(Bforb.shape)
    
    for k in range(NUM_MODALITIES):
        x_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(x_future[:,k * NUM_TS: k * NUM_TS + ts + 1])
        mask_input[:,k * NUM_TS: k * NUM_TS + ts + 1] = np.copy(Bforb[:,k * NUM_TS: k * NUM_TS + ts + 1])
    
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input)
    ts_rep = np.repeat(ts, x_input.shape[0]).reshape(-1, 1)
    pred = classifier.predict(np.concatenate([x_input * mask_input, ts_rep], axis=-1), verbose=1)
    y_pred[:,ts,:] = pred

print("computing loss...")    
y_pred = torch.Tensor(y_pred)
y_future = torch.Tensor(y_future)
cost = torch.zeros(Bforb.shape[0])
for m in range(NUM_MODALITIES):
    modality_cost = 1 if m in [0, 1] else 0.5
    cost += Bforb[:, m * NUM_TS: (m + 1) * NUM_TS].sum(1) * acquisition_cost * modality_cost
loss_future = compute_accumulated_loss(y_pred, y_future, loss_function) + cost 


computing predictions...
0
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 78s 394us/step
1
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 77s 389us/step
2
198619/198619 ━━━━━━━━━━━━━━━━━━━━ 78s 394us/step
3
 62848/198619 ━━━━━━━━━━━━━━━━━━━━ 51s 383us/step

KeyboardInterrupt: 

In [15]:
masksper = 256 
d = train_x_ts.shape[1] - 1
X_class = np.concatenate([train_x_ts]*masksper, 0)
Y_class = np.concatenate([train_y_ts]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)

zero_mask = X_class[:,:d] == 0
B[zero_mask] = 0
# remove 0 mask
mask_nonzero = np.sum(B, axis=1) != 0
B = B[mask_nonzero]
X_class = X_class[mask_nonzero]
Y_class = Y_class[mask_nonzero]

X_class[:,:d] = X_class[:,:d] * B
X_class.shape, Y_class.shape, B.shape

[758316. 758987. 758008. 757953. 757620. 757955. 757972. 758435. 757102.
 756993. 757385. 757279. 757917. 757722. 757728. 757501. 757812. 758526.
 758420. 757669. 758158. 758848. 758452. 758176. 757722. 758034. 758743.
 759001. 758049. 757567. 758648. 758536. 757489. 756505. 757903. 757405.
 757466. 757965. 758633. 757916. 758311. 757459. 758096. 757114. 757111.
 757801. 758337. 757861.]


((1418396, 49), (1418396, 3), (1418396, 48))

In [ ]:
ts_current = np.random.randint(0, num_ts, NUM_MASKS_CURRENT)
b_current = []
for i in range(NUM_MASKS_CURRENT):
    mask_temp = []
    for j in range(NUM_MODALITIES): 
        mask = np.sum(np.random.permutation(np.eye(NUM_TS))[:,:ts_current[i]+1][:, :np.random.randint(ts_current[i]+1)], 1)
        mask_temp.append(mask)
    b_current.append(mask_temp)
    
b_current = np.array(b_current)
b_current = np.transpose(b_current, (1, 0, 2)).reshape(-1, NUM_MODALITIES * NUM_TS)
b_empty = np.sum(b_current, 1) == 0
b_current = b_current[~b_empty]

In [7]:
masksper = 5
d = val_x_ts.shape[1]
X_class_val = np.concatenate([val_x_ts]*masksper, 0)
Y_class_val = np.concatenate([val_y_ts]*masksper, 0)
B_val = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class_val.shape[0])],
1)
B_val = np.float32(B_val.T)

# B_val = np.ones_like(B_val)

zero_mask_val = X_class_val[:,:d] == 0
B_val[zero_mask_val] = 0

# remove 0 mask
mask_nonzero_val = np.sum(B_val, axis=1) != 0
B_val = B_val[mask_nonzero_val]
X_class_val = X_class_val[mask_nonzero_val]
Y_class_val = Y_class_val[mask_nonzero_val]

X_class_val[:,:d] = X_class_val[:,:d] * B_val
X_class_val.shape, Y_class_val.shape, B_val.shape

((5595, 49), (5595, 3), (5595, 49))

In [10]:
Y_class.shape

(1132588, 3)

In [ ]:
y = np.argmax(logits, axis=1).astype(np.int32)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.metrics import AUC

TOTAL_FEATURES = train_x_ts.shape[-1]
HIDDEN_DIM = 10
NUM_CLASSES = train_y_ts.shape[-1]
DROPOUT_RATE = 0.75

def create_mlp():
    model = models.Sequential([
        layers.Input(shape=(TOTAL_FEATURES,)),
        layers.Dense(HIDDEN_DIM, activation='relu'),
        # layers.Dropout(DROPOUT_RATE),
        layers.Dense(HIDDEN_DIM, activation='relu'),
        # layers.Dropout(DROPOUT_RATE),
        layers.Dense(NUM_CLASSES, activation='softmax')  
    ])
    return model



# Compile the model
model = create_mlp()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(name='roc_auc'), AUC(name='prc', curve='PR')]
)

def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    checkpoint = ModelCheckpoint(
        '/work/users/d/d/ddinh/aaco/models/mlp_embedding.keras',  
        monitor='val_roc_auc',
        mode='max',  
        save_best_only=True,
        verbose=1
    )
    early_stopping = EarlyStopping(
        monitor='val_roc_auc', 
        mode='max', 
        patience=10,  
        verbose=1
    )
    with tf.device('/device:GPU:0'):
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[checkpoint, early_stopping],
            verbose=1
        )
    return history

def evaluate_model(model, X, y):
    loss, accuracy, roc_auc, prc = model.evaluate(X, y, verbose=0)
    print(f"Loss: {loss}, Accuracy: {accuracy}, ROC AUC: {roc_auc}, PRC: {prc}")
    return loss, accuracy, roc_auc

# history = train_model(model, train_x_ts, train_y_ts, val_x_ts, val_y_ts)
# evaluate_model(model, val_x_ts, val_y_ts)
history = train_model(model, X_class, Y_class, X_class_val, Y_class_val)
evaluate_model(model, X_class_val, Y_class_val)

def plot_history(history):
    plt.plot(history.history['roc_auc'])
    plt.plot(history.history['val_roc_auc'])
    plt.title('model roc_auc')
    plt.ylabel('roc_auc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['prc'])
    plt.plot(history.history['val_prc'])
    plt.title('model prc')
    plt.ylabel('prc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
plot_history(history)
#  val_prc: 0.5998 - val_roc_auc: 0.7917

Epoch 1/100
35373/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.6022 - loss: 0.7826 - prc: 0.6291 - roc_auc: 0.8070
Epoch 1: val_roc_auc improved from -inf to 0.76294, saving model to /work/users/d/d/ddinh/aaco/models/mlp_test.keras
35377/35377 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.6022 - loss: 0.7826 - prc: 0.6291 - roc_auc: 0.8070 - val_accuracy: 0.5645 - val_loss: 0.9315 - val_prc: 0.5431 - val_roc_auc: 0.7629
Epoch 2/100
35363/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6201 - loss: 0.7467 - prc: 0.6643 - roc_auc: 0.8241
Epoch 2: val_roc_auc improved from 0.76294 to 0.76604, saving model to /work/users/d/d/ddinh/aaco/models/mlp_test.keras
35377/35377 ━━━━━━━━━━━━━━━━━━━━ 45s 1ms/step - accuracy: 0.6201 - loss: 0.7467 - prc: 0.6643 - roc_auc: 0.8241 - val_accuracy: 0.5657 - val_loss: 0.9197 - val_prc: 0.5514 - val_roc_auc: 0.7660
Epoch 3/100
35340/35377 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.6223 - loss: 0.7426 - prc: 0.6686 - roc_auc: 0.8261
Ep

KeyboardInterrupt: 

My version

In [ ]:

dataset = load_adni_data()
x = dataset.x
y = dataset.y

mask_nan = np.isnan(x)

x[mask_nan] = 0

num_ts = y.shape[1]


In [ ]:
for i in range(num_ts):
    count = [0,0,0]
    total = 0
    for j in range(y.shape[0]):
        for k in range(3):
            if y[j,i,k] == 1:
                count[k] += 1
                total += 1
    count = np.array(count)
    print(count/total)

[0.06387226 0.64271457 0.29341317]
[0.08805668 0.61234818 0.29959514]
[0.10761421 0.59187817 0.30050761]
[0.125      0.58974359 0.28525641]
[0.16613757 0.54179894 0.29206349]
[0.13573883 0.56872852 0.29553265]
[0.178125  0.5796875 0.2421875]
[0.18283582 0.53731343 0.27985075]
[0.17350746 0.46455224 0.3619403 ]
[0.18235294 0.45294118 0.36470588]
[0.18596491 0.49473684 0.31929825]
[0.17098446 0.43005181 0.39896373]


In [14]:
count = [0,0,0]
total = 0
for i in range(num_ts):
    for j in range(y.shape[0]):
        for k in range(3):
            if y[j,i,k] == 1:
                count[k] += 1
                total += 1
count = np.array(count)
print(count/total)

[0.13054449 0.56998672 0.29946879]


In [4]:
mask_nan = x==0
# get the number of nan values
total = mask_nan.sum()
probability = total / x.size
probability

0.40976380572188953

In [5]:
# split dataset into train and test sets 80/20 with random seed 42
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((801, 12, 4), (201, 12, 4), (801, 12, 3), (201, 12, 3))

In [5]:
# x_train = prepare_time_series(torch.Tensor([range(0,12)] * x_train.shape[0]), torch.Tensor(x_train))[1]
# y_train = prepare_time_series(torch.Tensor([range(0,12)] * y_train.shape[0]), torch.Tensor(y_train))[1]

In [6]:
# x_train = torch.cat([x_train, x_train[:, [-1]]], dim=-2).numpy()
# y_train = torch.cat([y_train, y_train[:, [-1]]], dim=-2).numpy()

In [7]:
# y_train = np.round(y_train)

In [8]:
# x_ts = []
# y_ts = []

# for i in range(num_ts): 
#     x_temp = np.zeros_like(x_train)
#     x_temp[:, :i+1, :] = x_train[:, :i+1, :]
#     x_ts.append(x_temp)    
#     y_ts.append(y_train)
# x_ts = np.concatenate(x_ts, axis=0)
# y_ts = np.concatenate(y_ts, axis=0)
# x_ts.shape, y_ts.shape

In [6]:
"""temp
"""
x_ts = x_train
y_ts = y_train

In [7]:
x_all = []
for item in x_ts:
    x_all.append(item.flatten('F'))
x_train = np.array(x_all)
y_train = y_ts

In [ ]:
import numpy as np

def apply_masks(inputs, pm=0.2, pd_m=0.4):
    inputs = np.copy(inputs).reshape(-1, 12, 4)
    n, T, M = inputs.shape
    masked_inputs = inputs.copy()

    mask_m1 = np.random.binomial(1, pm, size=(n, T, M))
    
    t_max = np.random.randint(0, T, size=n)
    mask_m2 = np.ones((n, T, M))
    for i in range(n):
        mask_m2[i, t_max[i]:] = 0
    
    modality_drop = np.random.binomial(1, pd_m, size=(n, M))
    mask_m3 = np.ones((n, T, M))
    for m in range(M):
        mask_m3[:, :, m] *= modality_drop[:, m].reshape(-1, 1)

    final_mask = mask_m1 * mask_m2 * mask_m3
    masked_inputs *= final_mask

    return masked_inputs, final_mask.reshape(n, -1)

pm = 0.2
pd_m = 0.4 

In [ ]:
masksper = 256 * 2
d = x_train.shape[1]
X_class = np.concatenate([x_train]*masksper, 0)
Y_class = np.concatenate([y_train]*masksper, 0)
# B = np.concatenate(
# [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
# 1)
# B = np.float32(B.T)

_, B = apply_masks(X_class, pm, pd_m)
    
"""
remove for interpolation
"""
zero_mask = X_class == 0
B[zero_mask] = 0

# # remove 0 mask
mask_nonzero = np.sum(B, axis=1) != 0
B = B[mask_nonzero]
X_class = X_class[mask_nonzero]
Y_class = Y_class[mask_nonzero]


In [12]:
# mask_nan = X_class == 0
# B[mask_nan] = 0

In [16]:
X_class = np.concatenate((X_class*B, B), 1)

In [17]:
X_class.shape, Y_class.shape

((213901, 96), (213901, 12, 3))

In [18]:
location = []
for i in range(B.shape[0]):
    b = B[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location.append(temp)
    
location = np.array(location)

In [19]:
unique, counts = np.unique(location, return_counts=True)
print(np.asarray((unique, counts)).T)

[[    0 16433]
 [    1 22512]
 [    2 24842]
 [    3 30042]
 [    4 27878]
 [    5 18594]
 [    6 21267]
 [    7  9046]
 [    8 19186]
 [    9  5576]
 [   10 10502]
 [   11  8023]]


In [20]:
y_temp = []
for i in range(Y_class.shape[0]):
    y_location = Y_class[i, location[i]]
    mask_nan = np.isnan(y_location)
    y_location[mask_nan] = 0
    y_temp.append(y_location)
y_temp = np.array(y_temp)

In [21]:
# remove y with nan values
mask_zero = np.sum(y_temp, axis=1) != 0
X_class = X_class[mask_zero]
Y_class = y_temp[mask_zero]

In [22]:
Y_class = np.argmax(Y_class, axis=1)

In [23]:
# Train classifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:37:46] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:37:46] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=256, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# est.save_model('/work/users/d/d/ddinh/aaco/models/adni_different_masking.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:39:02] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [25]:
x_ts_val = []
y_ts_val = []

for i in range(num_ts): 
    x_temp = np.zeros_like(x_val)
    x_temp[:, :i+1, :] = x_val[:, :i+1, :]
    x_ts_val.append(x_temp)    
    y_ts_val.append(y_val)
x_ts_val = np.concatenate(x_ts_val, axis=0)
y_ts_val = np.concatenate(y_ts_val, axis=0)
x_ts_val.shape, y_ts_val.shape

((2412, 12, 4), (2412, 12, 3))

In [26]:
x_all_val = []
for item in x_ts_val:
    x_all_val.append(item.flatten('F'))
x_val = np.array(x_all_val)
y_val = y_ts_val

In [27]:
masksper = 1
d = x_val.shape[1]
X_class_val = np.concatenate([x_val]*masksper, 0)
Y_class_val = np.concatenate([y_val]*masksper, 0)
B_val = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class_val.shape[0])],
1)
B_val = np.float32(B_val.T)

# B_val = np.ones_like(B_val)

zero_mask_val = X_class_val == 0
B_val[zero_mask_val] = 0

# remove 0 mask
mask_nonzero_val = np.sum(B_val, axis=1) != 0
B_val = B_val[mask_nonzero_val]
X_class_val = X_class_val[mask_nonzero_val]
Y_class_val = Y_class_val[mask_nonzero_val]

In [28]:
location_val = []
for i in range(B_val.shape[0]):
    b = B_val[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location_val.append(temp)
    
location_val = np.array(location_val)

In [29]:
y_temp_val = []
for i in range(Y_class_val.shape[0]):
    y_location_val = Y_class_val[i, location_val[i]]
    mask_nan_val = np.isnan(y_location_val)
    y_location_val[mask_nan_val] = 0
    y_temp_val.append(y_location_val)
y_temp_val = np.array(y_temp_val)

In [30]:
# remove y with nan values for validation set
mask_zero_val = np.sum(y_temp_val, axis=1) != 0
X_class_val = X_class_val[mask_zero_val]
Y_class_val = y_temp_val[mask_zero_val]
B_val = B_val[mask_zero_val]

In [31]:
mask_zero_val.shape

(2246,)

In [32]:
X_class_val = np.concatenate((X_class_val*B_val, B_val), 1)

In [33]:
val_preds = est.predict_proba(X_class_val) 

In [34]:
print(np.mean(np.argmax(val_preds, 1)==np.argmax(Y_class_val, 1)))

0.5409617097061442


In [35]:
# this is temoprary evaluation, since it's not really correct.
# it should be evaluated on the accumulated predictions, not the individual predictions like this rolling out 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

pr_auc_macro = average_precision_score(Y_class_val, val_preds, average="macro")
print(pr_auc_macro)

roc_auc_macro = roc_auc_score(Y_class_val, val_preds, average="macro", multi_class="ovr")
print(roc_auc_macro)

0.5109105973299515
0.7030657167468642


In [36]:
np.sum(B_val.mean(0))

10.037846

In [ ]:
# baseline
# 0.5057228523811975
# 0.6798018971723648

# masking from a2mt paper
# 0.5109105973299515
# 0.7030657167468642